In [ ]:
!pip3 install pandas
!pip3 install sqlalchemy
!pip3 install psycopg2  # For PostgreSQL
!pip3 install pymysql   # For MySQL
!pip3 install cx_Oracle # For Oracle 
!pip3 install pyodbc

In [ ]:
import pyodbc
import pandas as pd

# Define your connection string
connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=Rahul\MSSQLSERVER01;"
    "DATABASE=ECC60jkl_HACK;"
    "Trusted_Connection=yes;"
)

# Establish the connection
connection = pyodbc.connect(connection_string)

# Use the connection to query the database with pandas
source = pd.read_sql("SELECT * FROM dbo.A017", connection)

# Print the dataframe
print(source)



In [ ]:
!pip3 install dask

In [ ]:
import pandas as pd
import pyodbc

# Define the connection string for the target database (Oracle EBS)
target_connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=Rahul\MSSQLSERVER01;"
    "DATABASE=ORACLE_EBS_HACK;"
    "Trusted_Connection=yes;"
)

# Establish the connection
connection = pyodbc.connect(target_connection_string)

# Use the connection to query the database with pandas
source = pd.read_sql("SELECT * FROM APPLSYS_FND_FLEX_VALUES", connection)

# Print the dataframe
print(source)